In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import json
import gensim.downloader as api
from sklearn.metrics import f1_score
import numpy as np
from gensim.models import KeyedVectors

In [2]:
with open('Json Task1/train_processed.json', 'r') as f:
    task1_train_data = json.load(f)

with open('Json Task1/val_processed.json', 'r') as f:
    task1_val_data = json.load(f)

with open('Json Task1/test_processed.json', 'r') as f:
    task1_test_data = json.load(f)

with open('Json Task2/train_processed.json', 'r') as f:
    task2_train_data = json.load(f)

with open('Json Task2/val_processed.json', 'r') as f:
    task2_val_data = json.load(f)

with open('Json Task2/test_processed.json', 'r') as f:
    task2_test_data = json.load(f)

bio_mapping_task1 = {'B_ORG': 0, 'I_ORG': 1, 'B_RESPONDENT': 2, 'I_RESPONDENT': 3, 'B_JUDGE': 4, 'I_JUDGE': 5,
               'B_STATUTE': 6, 'I_STATUTE': 7, 'B_OTHER_PERSON': 8, 'I_OTHER_PERSON': 9, 'B_COURT': 10, 'I_COURT': 11,
               'B_GPE': 12, 'I_GPE': 13, 'B_PETITIONER': 14, 'I_PETITIONER': 15, 'B_WITNESS': 16, 'I_WITNESS': 17,
               'B_CASE_NUMBER': 18, 'I_CASE_NUMBER': 19, 'B_PRECEDENT': 20, 'I_PRECEDENT': 21, 'B_DATE': 22, 'I_DATE': 23,
               'B_PROVISION': 24, 'I_PROVISION': 25, 'O': 26}
bio_mapping_task2 = {'O' : 0, 'I' : 1, 'B' : 2}

In [3]:
word2vec_model = api.load('word2vec-google-news-300')


In [4]:

word2vec = torch.FloatTensor(word2vec_model.vectors)
word2vec_u_ = np.concatenate((word2vec, np.zeros((1, 300), dtype = 'float32')), axis = 0)

# Add an unknown token to the vocabulary
word_to_index = {word: index for index, word in enumerate(word2vec_model.index_to_key)}
word_to_index['<unk>'] = len(word_to_index)

# Example usage
unknown_token_index = word_to_index['<unk>']

In [37]:
# from gensim.models import Word2Vec

# w2vModel = Word2Vec(vector_size=300, min_count=0)

# # Define your corpus
# my_corpus = [["I", "am", "nalish"]]

# # Train the Word2Vec model
# w2vModel.build_vocab(my_corpus)
# w2vModel.train(my_corpus, total_examples=len(my_corpus), epochs=10)
# w2vModel.wv["I"]


(1, 30)

In [5]:
# input_sequence = task1_train_data[str(0)]['text'].split(' ')
# sentence_embeddings = []
# sentence_embeddings = [word2vec_u_[word_to_index.get(word, word_to_index['<unk>'])] for word in input_sequence]
# print(type(sentence_embeddings), type(sentence_embeddings[0][0]))
# print(torch.tensor(sentence_embeddings))

<class 'list'> <class 'numpy.float32'>
tensor([[-0.0747, -0.0840,  0.1045,  ..., -0.0152, -0.0417, -0.0376],
        [ 0.0232, -0.1245,  0.0645,  ...,  0.1445,  0.2217,  0.0481],
        [-0.1738, -0.1846,  0.0199,  ..., -0.1709, -0.0233, -0.2656],
        ...,
        [ 0.1074,  0.0127,  0.3281,  ..., -0.0175, -0.1455, -0.1729],
        [ 0.1060,  0.0204,  0.2617,  ..., -0.3066,  0.1094, -0.1650],
        [ 0.0085,  0.2070,  0.0045,  ..., -0.0080,  0.2451, -0.2383]])


/var/folders/4f/kjgrgc293tn7cn556xk1h1g40000gn/T/ipykernel_4267/902827567.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  print(torch.tensor(sentence_embeddings))


In [3]:
import io

def load_fast_text_embeddings(fname):
    with io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore') as fin:
        n, d = map(int, fin.readline().split())
        data = {}
        
        # Save the current position in the file
        current_position = fin.tell()
        
        line_count = sum(1 for line in fin)
        
        fin.seek(current_position)
        
        for line in fin:
            tokens = line.rstrip().split(' ')
            data[tokens[0]] = list(map(float, tokens[1:]))
            
            line_count -= 1
            
            # print(f"Lines left: {line_count}")
            
    return data

# Adjust the path based on your downloaded file
ft_file_path = 'wiki-news-300d-1M-subword.vec'  # Adjust the path based on your downloaded file
fast_text_embeddings = load_fast_text_embeddings(ft_file_path)


In [4]:
fasttext_model_path = "wiki-news-300d-1M-subword.vec"
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path, binary=False)

In [5]:
def get_word_embedding(word):
    if word in fasttext_model:
        # If the word is in vocabulary, return its embedding
        return fasttext_model[word]
    else:
        # Decompose OOV word into subwords and aggregate subword embeddings
        subword_embeddings = [fasttext_model[subword] for subword in fasttext_model.key_to_index if subword in word]
        if subword_embeddings:
            # Aggregate subword embeddings
            return np.mean(subword_embeddings, axis=0)
        else:
            # If all subwords are OOV, return a zero vector
            return np.zeros(fasttext_model.vector_size)

In [6]:
ftt_cache_1 = {}
def get_word_vector():
    count = 0
    for key in task1_train_data:
        words = task1_train_data[key]['text'].split(' ')
        print(count)
        count +=1
        for word in words:
            if word not in ftt_cache_1:
                word_vector = get_word_embedding(word)
                ftt_cache_1[word] = word_vector

    for key in task2_train_data:
        print(count)
        count +=1
        words = task2_train_data[key]['text'].split(' ')
        for word in words:
            if word not in ftt_cache_1:
                word_vector = get_word_embedding(word)
                ftt_cache_1[word] = word_vector 

In [8]:
get_word_vector()  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
a = np.array(fast_text_embeddings['nalish'])
a.shape

KeyError: 'nalish'

In [ ]:
print()

In [9]:
def load_glove_embeddings(file_path):
    word_embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                values = line.split()
                word = values[0]
                vector = np.array(values[1:], dtype='float32')
                word_embeddings[word] = vector
            except ValueError as e:
                print(f"Error processing line: {line}\nError: {e}")
    return word_embeddings

glove_file_path = 'glove.840B.300d.txt'  # Adjust the path based on your downloaded file
glove_embeddings = load_glove_embeddings(glove_file_path)

Error processing line: . . . -0.1573 -0.29517 0.30453 -0.54773 0.098293 -0.1776 0.21662 0.19261 -0.21101 0.53788 -0.047755 0.40675 0.023592 -0.32814 0.046858 0.19367 0.25565 -0.021019 -0.15957 -0.1023 0.20303 -0.043333 0.11618 -0.18486 0.0011948 -0.052301 0.34587 0.052335 0.16774 -0.21384 0.055947 0.24934 -0.12179 0.16749 0.28922 -0.033739 0.3015 -0.13241 0.092635 0.37155 -0.2884 -0.0052731 -0.001005 -0.51153 -0.28476 -0.20139 0.11837 -0.0055891 0.43604 0.16796 -0.2701 0.063957 -0.093253 -0.22079 0.36501 0.06545 0.23941 -0.19292 0.098293 0.12172 -0.1168 -0.027436 0.20507 -0.39139 -0.23111 0.46239 0.22888 -0.028415 -0.1798 0.23817 0.28093 -0.47935 0.23177 -0.35587 0.14246 0.11861 0.011018 0.091986 0.0054809 -0.39955 -0.40183 -0.10629 -0.30851 0.12383 -0.16737 -0.43569 0.4211 -0.57416 -0.19964 0.51312 0.090747 -0.21657 0.043519 0.24288 0.081134 0.49104 -0.33342 -0.31056 -0.3136 0.26931 -0.14402 0.33185 -0.21662 -0.072985 0.080603 -0.7266 -0.098385 -0.36233 -0.25346 0.1154 0.25738 0.15802

In [13]:

class RNNModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(embedding_dim, 128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.rnn(x)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
class Task_data(Dataset):
    def __init__(self, data, bio_index, embedding_type):
        self.data = data
        self.length = len(self.data)
        self.bio_index =  bio_index
        self.embedding_type = embedding_type

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        input_sequence = self.data[str(index)]['text'].split(' ')
        sentence_embeddings = []
        if self.embedding_type == "glove":
            sentence_embeddings = [glove_embeddings.get(word, np.zeros(300, dtype = 'float32')) for word in input_sequence]
        elif self.embedding_type == "word2vec":
            sentence_embeddings = [word2vec_u_[word_to_index.get(word, word_to_index['<unk>'])] for word in input_sequence]
        elif self.embedding_type == "fast_text":
            sentence_embeddings = [np.array(ftt_cache_1.get(word, np.zeros(300, dtype = 'float32')), dtype = 'float32') for word in input_sequence]
        
        sentence_embeddings  = np.array(sentence_embeddings, dtype = 'float32')
        output_sequence = self.data[str(index)]['labels']
        output_labels = [self.bio_index[word] for word in output_sequence]

        return torch.tensor(sentence_embeddings), torch.tensor(output_labels)







In [14]:
def train_model(task, embedding_type, model, optimizer, criterion, device,  num_epochs = 12, batch_size = 1):
    train_dataloader  = None
    val_dataloader  = None
    test_dataloader = None

    if task == 1:
        train_dataloader = DataLoader(Task_data(task1_train_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=True)
        val_dataloader =  DataLoader(Task_data(task1_val_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=True)
    elif task == 2:
        train_dataloader = DataLoader(Task_data(task2_train_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=True)
        val_dataloader =  DataLoader(Task_data(task2_val_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=True)


    train_losses = []
    val_losses = []
    train_f1_scores = []
    val_f1_scores = []

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        total_train_loss = 0
        all_train_predictions = []
        all_train_targets = []

        for batch_idx, (inputs, targets) in enumerate(train_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            
            loss = 0
            for i in range(outputs.size(1)):  # Iterate over time steps
                loss += criterion(outputs[:, i, :], targets[:, i])  # Apply CrossEntropyLoss at each time step
            
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

            all_train_predictions.extend(outputs.argmax(dim=2).view(-1).cpu().numpy())
            all_train_targets.extend(targets.view(-1).cpu().numpy())


        avg_train_loss = total_train_loss / len(train_dataloader)
        train_losses.append(avg_train_loss)

        train_macro_f1 = f1_score(all_train_targets, all_train_predictions, average='macro')
        train_f1_scores.append(train_macro_f1)
        print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Training Macro F1-Score: {train_macro_f1}")

        model.eval()  # Set the model to evaluation mode
        total_val_loss = 0
        all_val_predictions = []
        all_val_targets = []

        with torch.no_grad():
            for val_inputs, val_targets in val_dataloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = model(val_inputs)

                loss = 0
                for i in range(val_outputs.size(1)):  # Iterate over time steps
                    loss += criterion(val_outputs[:, i, :], val_targets[:, i])  

                total_val_loss += loss.item()

                all_val_predictions.extend(val_outputs.argmax(dim=2).view(-1).cpu().numpy())
                all_val_targets.extend(val_targets.view(-1).cpu().numpy())

            avg_val_loss = total_val_loss / len(val_dataloader)
            val_losses.append(avg_val_loss)

            val_macro_f1 = f1_score(all_val_targets, all_val_predictions, average='macro')
            val_f1_scores.append(val_macro_f1)
        print(f"Epoch {epoch + 1},  Validation Loss: {avg_val_loss}, Validation Macro F1-Score: {val_macro_f1}")

    return train_losses, train_f1_scores, val_losses, val_f1_scores

def test_model(task, embedding_type, model, criterion, device, batch_size = 1):
    test_dataloader = None
    if task == 1:
        test_dataloader =  DataLoader(Task_data(task1_test_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=False)

    elif task == 2:
        test_dataloader =  DataLoader(Task_data(task2_test_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=False)

    total_test_loss = 0
    all_test_predictions = []
    all_test_targets = []

    with torch.no_grad():
        for test_inputs, test_targets in test_dataloader:
            test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            test_outputs = model(test_inputs)

            loss = 0
            for i in range(test_outputs.size(1)):  # Iterate over time steps
                loss += criterion(test_outputs[:, i, :], test_targets[:, i])  

            total_test_loss += loss.item()

            all_test_predictions.extend(test_outputs.argmax(dim=2).view(-1).cpu().numpy())
            all_test_targets.extend(test_targets.view(-1).cpu().numpy())

        avg_test_loss = total_test_loss / len(test_dataloader)
        test_macro_f1 = f1_score(all_test_targets, all_test_predictions, average='macro')
    print(f'Test Loss: {avg_test_loss}, Test Macro F1-Score: {test_macro_f1}')

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = RNNModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

train_model(task = 2, embedding_type = "fast_text", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 2, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

Epoch 1, Training Loss: 12.2881037281049, Training Macro F1-Score: 0.15568813170141665
Epoch 1,  Validation Loss: 7.432984331427099, Validation Macro F1-Score: 0.31532441302701336
Epoch 2, Training Loss: 6.377612185793996, Training Macro F1-Score: 0.35808379963699793
Epoch 2,  Validation Loss: 4.732505767584936, Validation Macro F1-Score: 0.5223734751226539
Epoch 3, Training Loss: 4.640554434778148, Training Macro F1-Score: 0.41904901004091455
Epoch 3,  Validation Loss: 3.8937513279860423, Validation Macro F1-Score: 0.5997386794288421
Epoch 4, Training Loss: 4.116977950858228, Training Macro F1-Score: 0.4854351699314682
Epoch 4,  Validation Loss: 3.719426327236167, Validation Macro F1-Score: 0.7138212785727606
Epoch 5, Training Loss: 3.7836438990866244, Training Macro F1-Score: 0.5165989631997503
Epoch 5,  Validation Loss: 3.5337905801798657, Validation Macro F1-Score: 0.6498762169183627
Epoch 6, Training Loss: 3.5502921895640047, Training Macro F1-Score: 0.535813571271462
Epoch 6,  Va

In [16]:
model = RNNModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_model(task = 1, embedding_type = "fast_text", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 1, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

Epoch 1, Training Loss: 19.218516604491718, Training Macro F1-Score: 0.0808433733799997
Epoch 1,  Validation Loss: 16.184011398940182, Validation Macro F1-Score: 0.141072664231955
Epoch 2, Training Loss: 13.153416865779912, Training Macro F1-Score: 0.22883408894437662
Epoch 2,  Validation Loss: 13.553429224633792, Validation Macro F1-Score: 0.26242899049319673
Epoch 3, Training Loss: 11.187542192484909, Training Macro F1-Score: 0.31665209101365893
Epoch 3,  Validation Loss: 12.602855301971756, Validation Macro F1-Score: 0.3142004253977762
Epoch 4, Training Loss: 10.205675638268575, Training Macro F1-Score: 0.3602823355607712
Epoch 4,  Validation Loss: 11.32415230548588, Validation Macro F1-Score: 0.3407392691276974
Epoch 5, Training Loss: 9.512964323561318, Training Macro F1-Score: 0.3880047217981208
Epoch 5,  Validation Loss: 10.784699052675656, Validation Macro F1-Score: 0.3718043403007735
Epoch 6, Training Loss: 9.016700840596249, Training Macro F1-Score: 0.4043438030756654
Epoch 6,

In [ ]:
model = RNNModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_model(task = 1, embedding_type = "glove", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 1, embedding_type = "glove", model = model, criterion=criterion, device=device)

In [ ]:
model = RNNModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_model(task = 2, embedding_type = "fast_text", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 2, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

In [ ]:
model = RNNModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_model(task = 2, embedding_type = "word2vec", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 2, embedding_type = "word2vec", model = model, criterion=criterion, device=device)

In [ ]:
model = RNNModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.CrossEntropyLoss()

train_model(task = 2, embedding_type = "glove", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 2, embedding_type = "glove", model = model, criterion=criterion, device=device)